In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)

from actionweaver.llms import patch
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable

from autocoder.bot import AutoCoder
from autocoder.rag import RepositoryIndex

In [3]:
github_repository = "TengHu/auto_coder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)

index = RepositoryIndex(github_api, github_repository)

Indexing codebase TengHu/auto_coder
Indexing autocoder/__init__.py
Indexing autocoder/bot.py
Indexing autocoder/pydantic_models.py
Indexing autocoder/rag.py
Indexing autocoder/telemetry.py
Indexing main.py


In [8]:
auto_coder = AutoCoder(github_api, index)

[System] Branch 'aw_demo_bot_v8' created successfully, and set as current active branch.


In [9]:
res = auto_coder("remove unused imports in autocoder/bot.py")
print(res)

["- No new files were created.\n- The following existing files received updates: \n  - 'Updated file autocoder/bot.py' (mentioned multiple times, possibly indicating multiple update attempts)\n  - 'Updated file autocoder/rag.py'\n- Issues encountered included:\n  - Updates to file content failed because the expected old content was not located. It might be beneficial to utilize the read_file action to retrieve the latest file contents before attempting updates. This issue was noted twice, suggesting repeated problems with updating file content."]


In [7]:
auto_coder("create a pr")

'The pull request has been successfully created with the number 31. It includes the changes to reorder the imports in `autocoder/bot.py` to follow PEP 8 guidelines for better code readability and standardization.'

In [11]:
nodes = index.query("AutoCoder")

In [21]:
nodes.sort(key=lambda n: n.score + n.metadata['loc'], reverse=True)


In [20]:
nodes[-1].metadata['loc']

221

In [22]:
res = auto_coder.search_code("AutoCoder")

In [23]:
print (res)

Showing top 4 of 4 results:
Filepath: `autocoder/bot.py`
File contents: import datetime
import os
import uuid
from typing import List, Union

from actionweaver import action
from actionweaver.utils.tokens import TokenUsageTracker
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable
from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.node_parser import CodeSplitter
from openai import AzureOpenAI, OpenAI
from pydantic import BaseModel, Field

from autocoder.pydantic_models import create_context, create_tasks
from autocoder.telemetry import trace_client

assert os.environ["MODEL"]
MODEL = os.environ["MODEL"]


DIVIDING_LINE = """
###############
This section is divider and not a part of the code.
{input}
#############

"""


class AutoCoder:
    def __init__(self, github_api, index):
        self.github_api = github_api

        # self.client = trace_client(AzureOpenAI(
        #     azure_endpoint = 

In [29]:
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
Function
)
messages = [{'role': 'system',
  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},
 {'role': 'user', 'content': 'Move trace_client function to bot.py.'},
 ChatCompletionMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{"description":"Move the \'trace_client\' function from its current location to the \'bot.py\' file."}', name='PlanCodeChange'), type='function')]),
 {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',
  'role': 'tool',
  'name': 'PlanCodeChange',
  'content': "- New files created: []\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\n        - Problems encountered: []\n        ###\n    Ignore others,    Make the previous message more user-friendly\n        "}
           ]

In [30]:
from openai import OpenAI


client = OpenAI()
response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages,
            stream=False,
        )

In [31]:
print (response)

ChatCompletion(id='chatcmpl-8h2sHwJwIR8SOkWxw2ypBGd80Uyom', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?', role='assistant', function_call=None, tool_calls=None))], created=1705269845, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=63, prompt_tokens=135, total_tokens=198))


In [32]:
print (response.choices[0].message.content)

I can assist you with moving the `trace_client` function to `bot.py`, but I need access to the source code or need to know where the current `trace_client` function is located. Can you provide the current code file that contains the `trace_client` function or tell me where to find it?


In [17]:
print (messages[0]['content'])

You are a coding assistant, you have the capability to assist with code-related tasks and modify files.


In [18]:
messages

[{'role': 'system',
  'content': 'You are a coding assistant, you have the capability to assist with code-related tasks and modify files.'},
 {'role': 'user', 'content': 'Move trace_client function to bot.py.'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9f2agXdE5nnUl1X04XkYHjeQ', function=Function(arguments='{"description":"Move the \'trace_client\' function from its current location to the \'bot.py\' file."}', name='PlanCodeChange'), type='function')]),
 {'tool_call_id': 'call_9f2agXdE5nnUl1X04XkYHjeQ',
  'role': 'tool',
  'name': 'PlanCodeChange',
  'content': "I've read code and attempted to create code changes. The outcome of these code changes is as follows:\n        - New files created: []\n        - Existing files updated: [['Updated file autocoder/bot.py', 'Updated file autocoder/telemetry.py']]\n        - Problems encountered: []\n        ###\n        Make the previous message more user-friendl